In [1]:
import json

json_file_path = "/home/zjusst/qms/webarena/result_stage_1_explore/trajs/812_1.json"
with open(json_file_path, "r") as f:
    trajs = json.load(f)

print(len(trajs))

10


In [11]:
trajs[8]

{'url_before': 'http://10.130.138.30:9999/',
 'url_after': 'http://10.130.138.30:9999/',
 'url_real_before': 'http://reddit.com/',
 'url_real_after': 'http://reddit.com/',
 'a11y_before': 'Tab 0 (current): Postmill\n\n[8045] RootWebArea \'Postmill\' focused: True\n\t[8251] HeaderAsNonLandmark \'\'\n\t\t[8252] link \'Home\'\n\t[8276] link \'Forums\'\n\t[8277] link \'Wiki\'\n\t[8285] searchbox \'Search query\'\n\t[8286] link \'Notifications (0)\'\n\t[8287] link \'Submit\'\n\t[8056] button \'MarvelsGrantMan136\' hasPopup: menu expanded: False\n\t[8058] main \'\'\n\t\t[8659] link \'Submissions\'\n\t\t[8660] link \'Comments\'\n\t\t[8062] button \'Filter on: Subscribed\' hasPopup: menu expanded: False\n\t\t[8064] button \'Sort by: Hot\' hasPopup: menu expanded: False\n\t\t[8065] article \'\'\n\t\t\t[8068] HeaderAsNonLandmark \'\'\n\t\t\t\t[8294] heading \'what is the recommended console to buy these days\'\n\t\t\t\t\t[8663] link \'what is the recommended console to buy these days\'\n\t\t\t\t

In [3]:
import re

def judge_node_same(obs_nodes_info_a, obs_nodes_info_b, threshold=3):
    """
    判断两个状态之间的节点差异是否超过阈值
    obs_nodes_info_a 和 b 都是一个字典，字典的key是节点id，value 是一个字典，例如：
    '11765': {'backend_id': 19269,
    'union_bound': [0.0, 0.0, 10.0, 10.0],
    'text': "[11765] RootWebArea 'Postmill' focused: True"}

    对于 obs_nodes_info_a 的一个元素，如果 obs_nodes_info_b 中存在一个元素，其 union_bound 一致，并且 text 去掉 [xxx] 后一致，则认为两个元素是相同的
    如果不同的元素小于 threshold，则认为两个状态是相同的
    这里所谓的 key 和 backend_id 都不重要，不作为参考依据

    返回值：
    - 如果两个状态差异小于等于 threshold，相同，返回 True
    - 如果两个状态差异超过阈值，返回 False
    """

    diff_count = 0

    b_nodes_set = set()
    for node_id, node_info in obs_nodes_info_b.items():
        union_bound = node_info['union_bound']
        cleaned_text = re.sub(r'\[.*?\]', '', node_info['text'])
        b_nodes_set.add(f"{union_bound}_{cleaned_text}")

    for node_id, node_info in obs_nodes_info_a.items():
        union_bound = node_info['union_bound']
        cleaned_text = re.sub(r'\[.*?\]', '', node_info['text'])
        if f"{union_bound}_{cleaned_text}" not in b_nodes_set:
            diff_count += 1
    # print(diff_count)
    return diff_count <= threshold

In [4]:
def a11y_to_components(a11y):
    """
    将 a11y 转换为组件列表
    """
    # 对于每一行
    # 如果包含 [xxx]，则去掉 [xxx]，将后面的内容视作一个组件

    components = []
    for line in a11y.split("\n"):
        # 去掉前后空格
        line = line.strip()
        if line == "":
            continue
        match = re.match(r'\[(\d+)\]', line)
        if match:
            # component_id = match.group(1)
            component_content = line[match.end():].strip()
            components.append(component_content)
        else:
            components.append(line)
    return components


In [5]:
def judge_node_same_from_a11y(a11y_a, a11y_b, threshold=3):
    """
    判断两个状态之间的节点差异是否超过阈值
    a11y_a 和 b 都是一个字符串，字符串的格式为例如：
    [5207] heading 'relationship_advice — relationship_advice'
        [5209] link 'relationship_advice — relationship_advice'
    [5215] button 'Subscribe No subscribers'
        [5571] generic 'No subscribers'
    [5216] StaticText '5,721 submissions'

    首先去掉前面的 [xxx]，将后面的内容视作一个组件

    对于 a11y_a 的一个组件，如果 a11y_b 中存在一个组件，其组件内容一致，则认为两个组件是相同的
    如果不同的组件小于 threshold，则认为两个状态是相同的

    返回值：
    - 如果两个状态差异小于等于 threshold，相同，返回 True
    - 如果两个状态差异超过阈值，返回 False
    """

    diff_count = 0

    a_components = a11y_to_components(a11y_a)
    b_components = a11y_to_components(a11y_b)

    for a_component in a_components:
        if a_component not in b_components:
            diff_count += 1
    # print(diff_count)
    return diff_count <= threshold

In [6]:
judge_node_same_from_a11y(trajs[0]["a11y_before"], trajs[2]["a11y_after"])

True

In [7]:
from rank_bm25 import BM25Okapi
def bm25_retrieval(query, corpus, top_n=3):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.split(" ")
    top_n_docs = bm25.get_top_n(tokenized_query, corpus, n=top_n)
    top_n_docs_index = [corpus.index(doc) for doc in top_n_docs]
    return top_n_docs_index

In [8]:
# judge_node_difference(data[0]["state_before"]['text']['obs_nodes_info'], data[2]["state_after"]['text']['obs_nodes_info'])
# bm25_retrieval('a', ['bcc', 'a v'])

In [ ]:
import os

# /home/zjusst/qms/webarena/result_stage_1_explore/trajs 底下的所有文件名
file_names = os.listdir("/home/zjusst/qms/webarena/result_stage_1_explore/trajs")
file_names = [file_name for file_name in file_names if file_name.endswith(".json")]
# file_names.sort() 自定义排序，按照 _ 分割，取后面的数字排序
file_names.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))
len(file_names)


['812_1.json',
 '812_2.json',
 '812_3.json',
 '812_4.json',
 '812_5.json',
 '812_6.json',
 '812_7.json',
 '812_8.json',
 '812_9.json',
 '812_10.json',
 '812_11.json',
 '812_12.json',
 '812_13.json',
 '812_14.json',
 '812_15.json',
 '812_16.json',
 '812_17.json',
 '812_18.json',
 '812_19.json',
 '812_20.json',
 '812_21.json',
 '812_22.json',
 '812_23.json',
 '812_24.json',
 '812_25.json',
 '812_26.json',
 '812_27.json',
 '812_28.json',
 '812_29.json',
 '812_30.json',
 '812_31.json',
 '812_32.json',
 '812_33.json',
 '812_34.json',
 '812_35.json',
 '812_36.json',
 '812_37.json',
 '812_38.json',
 '812_39.json',
 '812_40.json',
 '812_41.json',
 '812_42.json',
 '812_43.json',
 '812_44.json',
 '812_45.json',
 '812_46.json',
 '812_47.json',
 '812_48.json',
 '812_49.json',
 '812_50.json',
 '812_51.json',
 '812_52.json',
 '812_53.json',
 '812_54.json',
 '812_55.json',
 '812_56.json',
 '812_57.json',
 '812_58.json',
 '812_59.json',
 '812_60.json',
 '812_61.json',
 '812_62.json',
 '812_63.json',
 

In [21]:
from tqdm import tqdm

unique_page_ids = []  # (file_name, traj_idx, 0|1代表前还是后, real_url) 
unique_page_a11ys = []
#unique_page_nodes = []
triplets = []  # (page_id, action, page_id)

for file_name in tqdm(file_names):
    with open(os.path.join("/home/zjusst/qms/webarena/result_stage_1_explore/trajs", file_name), "r") as f:
        trajs = json.load(f)

    for traj_idx, traj in enumerate(trajs):
        if len(unique_page_ids) == 0:
            unique_page_ids.append((file_name, traj_idx, 0, traj['url_real_before']))
            unique_page_a11ys.append(traj["a11y_before"])
            #unique_page_nodes.append(traj["state_before"]['text']['obs_nodes_info'])

        prev_page_id = -1
        # 从 unique_page_a11ys 检索出最接近的三个
        top_n_docs_index = bm25_retrieval(traj["a11y_before"], unique_page_a11ys, 3)
        for doc_idx in top_n_docs_index:
            # if judge_node_same(traj["state_before"]['text']['obs_nodes_info'], unique_page_nodes[doc_idx], 3):
            #     prev_page_id = unique_page_ids[doc_idx]
            #     break
            if judge_node_same_from_a11y(traj["a11y_before"], unique_page_a11ys[doc_idx], 3):
                prev_page_id = unique_page_ids[doc_idx]
                break
        if prev_page_id == -1:
            unique_page_ids.append((file_name, traj_idx, 0, traj['url_real_before']))
            unique_page_a11ys.append(traj["a11y_before"])
            #unique_page_nodes.append(traj["state_before"]['text']['obs_nodes_info'])
            prev_page_id = unique_page_ids[-1]

        action_str = traj["action_str"]

        after_page_id = -1
        top_n_docs_index = bm25_retrieval(traj["a11y_after"], unique_page_a11ys, 3)
        for doc_idx in top_n_docs_index:
            # if judge_node_same(traj["state_after"]['text']['obs_nodes_info'], unique_page_nodes[doc_idx], 3):
            #     after_page_id = unique_page_ids[doc_idx]
            #     break
            if judge_node_same_from_a11y(traj["a11y_after"], unique_page_a11ys[doc_idx], 3):
                after_page_id = unique_page_ids[doc_idx]
                break
        if after_page_id == -1:
            unique_page_ids.append((file_name, traj_idx, 1, traj['url_real_after']))
            unique_page_a11ys.append(traj["a11y_after"])
            #unique_page_nodes.append(traj["state_after"]['text']['obs_nodes_info'])
            after_page_id = unique_page_ids[-1]
            
        triplets.append((prev_page_id, action_str, after_page_id))

print(len(triplets))


100%|██████████| 104/104 [00:32<00:00,  3.21it/s]

1025


In [22]:
len(unique_page_ids)

142

In [30]:
parsed_triplets = []
for triplet in triplets:
    temp_triplet = (
        f"{triplet[0][0]}_{triplet[0][1]}_{triplet[0][2]}_{triplet[0][3]}",
        triplet[1],
        f"{triplet[2][0]}_{triplet[2][1]}_{triplet[2][2]}_{triplet[2][3]}"
    )
    parsed_triplets.append(temp_triplet)
len(parsed_triplets)

1025

In [32]:
import networkx as nx
from pyvis.network import Network

# --------------------------------------------------------------------------
# 1. 你的三元组数据 (用示例数据代替)
# 在实际使用中，你会从文件中加载这些数据
# 格式: (前页面标识, 动作描述, 后页面标识)
# 页面标识可以是页面的URL、截图文件的哈希值或你定义的任何唯一ID
# triplets = [
#     ('HomePage', 'click("Login Button")', 'LoginPage'),
#     ('HomePage', 'click("Products Link")', 'ProductsPage'),
#     ('ProductsPage', 'click("Item 1")', 'Item1_DetailsPage'),
#     ('ProductsPage', 'click("Item 2")', 'Item2_DetailsPage'),
#     ('Item1_DetailsPage', 'click("Add to Cart")', 'ShoppingCartPage'),
#     ('Item2_DetailsPage', 'click("Add to Cart")', 'ShoppingCartPage'),
#     ('LoginPage', 'type("username & password")', 'HomePage'), # 登录后回到主页
#     ('ShoppingCartPage', 'go_back()', 'ProductsPage') # 从购物车返回
# ]
# --------------------------------------------------------------------------


# 2. 使用 NetworkX 创建一个有向图 (因为 A->B 和 B->A 是不同的)
G = nx.DiGraph()

# 3. 遍历三元组数据，填充图的节点和边
for source, action, target in parsed_triplets:
    # 添加节点 (如果节点已存在，NetworkX会自动忽略)
    # 我们可以为节点添加属性，例如'title'，当鼠标悬停时会显示
    G.add_node(source, title=f"Page ID: {source}")
    G.add_node(target, title=f"Page ID: {target}")

    # 添加边，并把动作描述作为边的标签
    G.add_edge(source, target, label=action)


# --------------------------------------------------------------------------
# 4. 使用 Pyvis 进行交互式可视化
# 创建一个Pyvis网络图对象
# notebook=True 如果你在Jupyter Notebook中使用
# directed=True 会让边带上箭头，非常重要
net = Network(height='2000px', width='100%', notebook=True, directed=True)

# 从 NetworkX 对象加载图数据
net.from_nx(G)

# 为可视化添加一些物理引擎的选项，让布局更好看
# net.toggle_physics(True)

# 手动设定物理引擎参数
net.set_options("""
{
  "physics": {
    "enabled": true,
    "solver": "forceAtlas2Based",
    "forceAtlas2Based": {
      "springLength": 600
    }
  }
}
""")

# 添加交互UI的选项，例如筛选节点
#net.show_buttons(filter_=['physics'])

# 5. 生成HTML文件
# 这个文件将保存在你的脚本所在的目录下
# 你可以直接在浏览器中打开它
net.show('interactive_graph.html')

print("成功生成 'interactive_graph.html' 文件！请在浏览器中打开查看。")

interactive_graph.html
成功生成 'interactive_graph.html' 文件！请在浏览器中打开查看。
